In [1]:
import numpy as np
import pandas as pd
import torch
import os
import joblib
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from collections import Counter

# 确保导入了您所有的配置、模型定义和工具函数
import config 
from models import LSTMFeatureExtractor 
from utility_uad_svm import load_data, create_sequences_for_svm
from utility_uad_svm import get_features, standardize_features # 注意：不再使用 complex_balance_data

--- [Config] 正在使用的设备: cpu ---


In [2]:
def simple_balance_data(X_feat, y_labels, random_state=42):
    """
    仅基于标签 (y_labels) 对特征进行欠采样，实现 LOS/NLOS 平衡。
    """
    print(f"  - 原始类别分布: {Counter(y_labels)}")
    
    # 找到 LOS (0) 和 NLOS (1) 的索引
    idx_los = np.where(y_labels == 0)[0]
    idx_nlos = np.where(y_labels == 1)[0]
    
    # 确定最小组大小
    min_size = min(len(idx_los), len(idx_nlos))
    
    if min_size == 0:
        print("警告: 某一类样本数为零，无法平衡。返回空数组。")
        return np.empty((0, X_feat.shape[1])), np.empty((0,))

    # 对索引进行欠采样
    np.random.seed(random_state)
    idx_los_bal = np.random.choice(idx_los, min_size, replace=False)
    idx_nlos_bal = np.random.choice(idx_nlos, min_size, replace=False)

    # 合并索引并打乱
    combined_idx = np.concatenate([idx_los_bal, idx_nlos_bal])
    np.random.shuffle(combined_idx)
    
    # 重新映射到特征和标签
    X_resampled = X_feat[combined_idx]
    y_resampled = y_labels[combined_idx]

    print(f"✅ 特征空间简单平衡完成。每类 {min_size} 条，总样本数: {len(X_resampled)}")
    return X_resampled, y_resampled

In [3]:
def clean_features(F_feat):
    """清除特征中的 NaN 和 Inf 值。"""
    nan_count = np.sum(np.isnan(F_feat))
    inf_count = np.sum(np.isinf(F_feat))
    
    if nan_count > 0 or inf_count > 0:
        print(f"【⚠️ 再次清理】发现 {nan_count} 个 NaN 和 {inf_count} 个 Inf。正在执行 nan_to_num...")
        # nan_to_num 会将 NaN 替换为 0，将 Inf 替换为数值极限
        F_feat = np.nan_to_num(F_feat, nan=0.0, posinf=1e8, neginf=-1e8) 
        print("清理完成。")
    return F_feat

In [4]:
device = torch.device(config.DEVICE)
SRC_IDS = config.SRC_IDS
TGT_ID = config.TGT_ID
CSV_PATHS = config.CSV_PATHS
MAX_SVM_SAMPLES = 20000 

In [5]:
from sklearn.preprocessing import StandardScaler
print(f"--- [SVM Main] 正在使用设备: {device} ---")
    
    # --- 1. 加载原始数据并标准化 ---
print("\n--- [步骤 1] 正在加载原始数据和执行原始数据标准化... ---")
df_src, df_tgt = load_data(SRC_IDS, TGT_ID, CSV_PATHS)
    
    # ！！ 关键修正：原始数据标准化 ！！

scaler = StandardScaler()
df_src_features = df_src[config.FEATURES].copy()
df_tgt_features = df_tgt[config.FEATURES].copy()

df_src[config.FEATURES] = scaler.fit_transform(df_src_features)
df_tgt[config.FEATURES] = scaler.transform(df_tgt_features)
print("✅ 原始数据标准化完成。")

--- [SVM Main] 正在使用设备: cpu ---

--- [步骤 1] 正在加载原始数据和执行原始数据标准化... ---
[load_data] 源域数据形状: (710063, 13), 目标域数据形状: (12614, 14)
✅ 原始数据标准化完成。


In [6]:
print(f"--- [SVM Main] 正在使用设备: {device} ---")
    
    # --- 1. 加载原始数据并标准化 ---
print("\n--- [步骤 1] 正在加载原始数据和执行原始数据标准化... ---")
df_src, df_tgt = load_data(SRC_IDS, TGT_ID, CSV_PATHS)
    
# ！！ 关键修正：原始数据标准化 ！！
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_src_features = df_src[config.FEATURES].copy()
df_tgt_features = df_tgt[config.FEATURES].copy()

df_src[config.FEATURES] = scaler.fit_transform(df_src_features)
df_tgt[config.FEATURES] = scaler.transform(df_tgt_features)
print("✅ 原始数据标准化完成。")

--- [SVM Main] 正在使用设备: cpu ---

--- [步骤 1] 正在加载原始数据和执行原始数据标准化... ---
[load_data] 源域数据形状: (710063, 13), 目标域数据形状: (12614, 14)
✅ 原始数据标准化完成。


In [7]:
# =============================================================================
# SVM 单专家主函数
# =============================================================================
def main_single_svm():
    # ... (与之前相同的全局配置加载) ...
    # --- 2. 创建序列 ---
    X_src, y_src, S_src, I_src_raw, X_tgt, y_tgt, S_tgt, TGT_INDICES = create_sequences_for_svm(df_src, df_tgt, config)
    if X_src is None or X_tgt is None: return

    # --- 3. 特征提取 (加载 G_f 模型) ---
    print("\n--- [步骤 3] 正在加载 G_f 模型并提取特征... ---")
    G_f = LSTMFeatureExtractor(
        input_size=config.LSTM_INPUT_SIZE, hidden_size=config.LSTM_HIDDEN_SIZE,
        num_layers=config.LSTM_NUM_LAYERS, dropout=config.LSTM_DROPOUT
    ).to(device)

    G_f_path = os.path.join(config.MODEL_SAVE_DIR, "G_f_final.pth")
    try:
        G_f.load_state_dict(torch.load(G_f_path, map_location=device))
        G_f.eval()
    except FileNotFoundError:
        print(f"❌ 错误：未找到 G_f 权重文件。请确保 DANN 训练已完成。")
        return

    F_src_raw = get_features(G_f, X_src, config.BATCH_SIZE, device)
    F_tgt_raw = get_features(G_f, X_tgt, config.BATCH_SIZE, device)

    # --- 4. 特征标准化 (对 256 维特征进行第二次标准化) ---
    F_src_scaled, F_tgt_scaled = standardize_features(F_src_raw, F_tgt_raw)

    # 强制清洗源域和目标域的缩放后特征
    F_src_scaled = clean_features(F_src_scaled)
    F_tgt_scaled = clean_features(F_tgt_scaled)    
        # =============================================================================
        # 5. 【核心修改】整体平衡和训练单一 SVM
        # =============================================================================
    print("\n--- [步骤 5] 整体平衡源域数据 (GPS + PL) 并训练单一 SVM ---")

    # 5a. 整体平衡源域特征
    F_bal, y_bal = simple_balance_data(F_src_scaled, y_src, random_state=42)

    # 5b. 二次欠采样 (控制规模)
    if len(F_bal) > MAX_SVM_SAMPLES:
        F_bal, _, y_bal, _ = train_test_split(F_bal, y_bal, train_size=MAX_SVM_SAMPLES, random_state=42, stratify=y_bal)
        print(f" - 训练集规模已从 {len(F_src_scaled)} 减少到 {len(F_bal)}。")
    else:
         print(f" - 训练集规模: {len(F_bal)}。无需二次采样。")
    
    # 5c. 训练单一 SVM 模型
    print("\n--- 训练单一全局 SVM 模型 ---")
    svm_global = SVC(kernel='rbf', C=0.5, random_state=42)
    if len(F_bal) > 0:
        with tqdm(total=1, desc="训练全局 SVM 模型", ncols=80, leave=False):
            svm_global.fit(F_bal, y_bal)
        print(f"✅ 全局 SVM 训练完成。训练样本数: {len(F_bal)}")
    else:
        print(" - 警告：SVM 样本不足，跳过训练。")

    # --- 6. 目标域预测 (单一模型预测所有特征) ---
    print("\n--- [步骤 6] 目标域预测 (单一模型)... ---")
    
    if 'svm_global' in locals() and len(F_tgt_scaled) > 0:
        with tqdm(total=len(F_tgt_scaled), desc="  - 全局模型预测中", leave=False) as pbar:
            y_pred_tgt = svm_global.predict(F_tgt_scaled)
            pbar.update(len(F_tgt_scaled))
    else:
        print("警告：无法预测，请检查模型是否训练成功或目标域数据量。")
        return

    # --- 7. 评估和保存结果 (与之前的逻辑相同) ---
    print("\n--- [步骤 7] 评估和报告 ---")
    # ... (评估代码与之前相同) ...
    if len(y_tgt) > 0 and not np.all(y_tgt == 0):
        accuracy = accuracy_score(y_tgt, y_pred_tgt)
        report = classification_report(y_tgt, y_pred_tgt, output_dict=False, zero_division=0)
        
        print("\n=======================================================")
        print("✅ DANN-LSTM 单一全局 SVM 性能报告：")
        print(f"目标域总体 SVM 准确率 (Accuracy): {accuracy:.4f}")
        print("目标域 SVM 评估报告:\n", report)
        print("=======================================================")
    else:
        print("\n✅ 预测完成。目标域无标签或标签为占位符，跳过评估。")

    # ... (保存预测结果到 CSV 的代码与之前相同) ...
    # =============================================================================
# 8. 预测结果回溯合并与保存
# =============================================================================
    print("\n--- [步骤 8] 预测结果回溯合并与保存 ---")

    # 检查所需变量是否存在，确保流程安全
    if 'df_tgt' in locals() and 'TGT_INDICES' in locals() and 'y_pred_tgt' in locals():
        
        # 1. 创建一个与原始 df_tgt 长度相同的空预测列
        # 使用 NaN 作为占位符，因为不是所有原始行都能形成一个序列的尾部。
        full_predictions = pd.Series(
            data=np.nan, 
            index=df_tgt.index, 
            dtype=float 
        )

        # 2. ！！关键的回溯操作：使用 TGT_INDICES 映射预测值 ！！
        # TGT_INDICES 包含了每个预测值对应的原始 df_tgt 的索引。
        full_predictions.loc[TGT_INDICES] = y_pred_tgt

        # 3. 将新的预测列附加到原始 DataFrame
        df_tgt['predicted_multipath'] = full_predictions

        # 4. 定义输出并保存文件
        OUTPUT_DIR = os.path.join(config.MODEL_SAVE_DIR, "results_single_svm")
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        OUTPUT_PATH = os.path.join(OUTPUT_DIR, "tgt_data_with_single_svm_predictions.csv")

        df_tgt.to_csv(OUTPUT_PATH, index=False)

        print(f"✅ 最终结果文件已成功保存到: {OUTPUT_PATH}")
        print("文件已包含所有原始列，并在末尾附加了 'predicted_multipath' 列。")
    else:
        print("警告：缺少 df_tgt、TGT_INDICES 或 y_pred_tgt 变量，跳过结果文件保存。")
# 运行主函数
if __name__ == '__main__':
    main_single_svm()


--- [序列制作] 正在创建 SVM 所需的序列... ---
  - 正在处理源域 (仿真) 数据...
  - X 形状: (709919, 10, 5), y 形状: (709919,), S 形状: (709919,)

  - 正在处理目标域 (真实) 数据 (无标签处理)...
  - X 形状: (12443, 10, 5), y 形状: (0,), S 形状: (12443,)
--- 序列制作完成 ---

--- [步骤 3] 正在加载 G_f 模型并提取特征... ---
  - 正在执行特征提取...


  - 提取特征中: 100%|██████████| 11093/11093 [00:52<00:00, 213.11it/s]


  - 特征提取完毕。输出形状: (709919, 128)
  - 正在执行特征提取...


  - 提取特征中: 100%|██████████| 195/195 [00:01<00:00, 170.57it/s]


  - 特征提取完毕。输出形状: (12443, 128)

--- [标准化] 正在对特征进行标准化... ---
  - 特征标准化完成 (基于源域统计量)。
【⚠️ 再次清理】发现 1240320 个 NaN 和 0 个 Inf。正在执行 nan_to_num...
清理完成。

--- [步骤 5] 整体平衡源域数据 (GPS + PL) 并训练单一 SVM ---
  - 原始类别分布: Counter({np.int64(1): 376554, np.int64(0): 333365})
✅ 特征空间简单平衡完成。每类 333365 条，总样本数: 666730
 - 训练集规模已从 709919 减少到 20000。

--- 训练单一全局 SVM 模型 ---


✅ 全局 SVM 训练完成。训练样本数: 20000

--- [步骤 6] 目标域预测 (单一模型)... ---



--- [步骤 7] 评估和报告 ---

✅ 预测完成。目标域无标签或标签为占位符，跳过评估。

--- [步骤 8] 预测结果回溯合并与保存 ---
警告：缺少 df_tgt、TGT_INDICES 或 y_pred_tgt 变量，跳过结果文件保存。
